# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1. Introduction: Business Problem <a name="introduction"></a>

In this project I will try to find the best location for an Italian Restaurant in Manhattan, NY. I will focus on locations/Neighborhoods where Italian Restaurants are 1st most common venue.



## 2. Data <a name="data"></a>

First, we need to retrieve the data for all neighborhoods located in our target area, Manhattan. After we retrieve the needed data, we will use the Foursquare API to identify the restaurants in the target area.

I will use the New York data in a json format in order to retrieve the coordinated of all neighborhoods in Manhattan. Data source: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json 

Using the Foursquare API, I retrieved the venues from all Manhattan neighborhoods. 


### Import necessary libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Let's download the New York data in a json format and read it.

In [6]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [8]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Tranform the data into a pandas dataframe

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
# loop through the data and fill the dataframe one row at a time.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

### Filter only Manhattan data as this is our focus area

In [11]:
# filter only Manhattan Borough

manhattan_data = neighborhoods[neighborhoods['Borough'].isin(['Manhattan'])]
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
6,Manhattan,Marble Hill,40.876551,-73.910660
100,Manhattan,Chinatown,40.715618,-73.994279
101,Manhattan,Washington Heights,40.851903,-73.936900
102,Manhattan,Inwood,40.867684,-73.921210
103,Manhattan,Hamilton Heights,40.823604,-73.949688


### Use geopy library to get the latitude and longitude values of Manhattan

In [12]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


### Visualize Manhattan neighborhoods

In [13]:

# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### Foursquare Credentials and Version

In [1]:
CLIENT_ID = '04O2SCC5QXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXLQKWCNT'
CLIENT_SECRET = 'NBJXMOXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXLVIG1VPUUC'
VERSION = '20180605'
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 04O2SCC5QXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXLQKWCNT
CLIENT_SECRET:NBJXMOXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXLVIG1VPUUC


### Explore Neighborhoods in Manhattan

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [18]:
# save data to csv
manhattan_venues.to_csv('manhattan_venues.csv',index=False)

### Filter only restaurants from venues

In [19]:
#filter restaurants
manhattan_restaurants = manhattan_venues[manhattan_venues['Venue Category'].str.contains("Restaurant")]
manhattan_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
8,Marble Hill,40.876551,-73.910660,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
17,Marble Hill,40.876551,-73.910660,Boston Market,40.877430,-73.905412,American Restaurant
28,Chinatown,40.715618,-73.994279,Kiki's,40.714476,-73.992036,Greek Restaurant
30,Chinatown,40.715618,-73.994279,Spicy Village,40.717010,-73.993530,Chinese Restaurant
32,Chinatown,40.715618,-73.994279,Wah Fung Number 1 Fast Food 華豐快餐店,40.717278,-73.994177,Chinese Restaurant
34,Chinatown,40.715618,-73.994279,Da Yu Hot Pot 大渝火锅,40.716735,-73.995752,Hotpot Restaurant
39,Chinatown,40.715618,-73.994279,Xi'an Famous Foods,40.715232,-73.997263,Chinese Restaurant
43,Chinatown,40.715618,-73.994279,Forgtmenot,40.714459,-73.991546,New American Restaurant
44,Chinatown,40.715618,-73.994279,Dimes,40.714830,-73.991719,American Restaurant
50,Chinatown,40.715618,-73.994279,Cervo's,40.714763,-73.991455,Spanish Restaurant


## 3. Methodology <a name="methodology"></a>

The objective of this project is to identify Neighborhoods where Italian Restaruants are 1st most common venues.

In the first step we have collected the New Your data which includes Borough, Neighborhood, Latitude and Longitude.

In the second step we will use the Foursquare API to identify the venues in Manhattan. We will focus only on Restaurants located in Manhattan.

In the final step we will cluster all restaurant locations using **k-means clustering** in order to identify the optimal Italian Restaurant location.

## 4. Analysis <a name="analysis"></a>

### Analyze Each Neighborhood

#### Convert categorical variable into dummy/indicator variables

In [20]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_restaurants[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_restaurants['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Persian Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
8,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,Marble Hill,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [81]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Persian Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Battery Park City,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.200000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.0,0.200000,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000,0.0000,0.200000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.200000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,Carnegie Hill,0.0,0.000000,0.062500,0.000000,0.0625,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.062500,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0625,0.0,0.187500,0.0,0.000000,0.0,0.000000,0.062500,0.000000,0.000000,0.0,0.062500,0.0,0.0,0.000000,0.0625,0.0,0.0,0.000000,0.0625,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.062500,0.062500,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.062500,0.0,0.0,0.000000,0.0,0.062500,0.0,0.0,0.000000,0.062500
2,Central Harlem,0.0,0.142857,0.142857,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.071429,0.142857,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.071429,0.0,0.0000,0.0,0.142857,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.071429,0.0,0.0,0.142857,0.000000,0.0,0.0,0.071429,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,Chelsea,0.0,0.000000,0.157895,0.052632,0.0000,0.052632,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.052632,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.157895,0.0,0.000000,0.0,0.000000,0.052632,0.052632,0.105263,0.0,0.052632,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.052632,0.0,0.052632,0.0,0.0,0.000000,0.000000,0.0,0.0,0.105263,0.000000,0.0,0.0,0.000000,0.000000,0.052632,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,Chinatown,0.0,0.000000,0.076923,0.000000,0.0000,0.051282,0.0,0.025641,0.0,0.0,0.025641,0.000000,0.179487,0.0,0.0,0.051282,0.051282,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.051282,0.0,0.051282,0.000000,0.000000,0.000000,0.0,0.025641,0.0,0.0,0.025641,0.0000,0.0,0.0,0.051282,0.0000,0.051282,0.0,0.0,0.0,0.0,0.025641,0.0,0.000000,0.0,0.0,0.025641,0.000000,0.0,0.0,0.000000,0.051282,0.0,0.0,0.000000,0.025641,0.000000,0.0,0.0,0.025641,0.0,0.025641,0.0,0.0,0.025641,0.076923


### Create the new dataframe and display the top 10 Restaurants for each neighborhood

In [82]:
def return_most_common_restaurants(row, num_top_restaurants):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_restaurants]

In [83]:
num_top_restaurants = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_restaurants):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_restaurants_sorted = pd.DataFrame(columns=columns)
neighborhoods_restaurants_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_restaurants_sorted.iloc[ind, 1:] = return_most_common_restaurants(manhattan_grouped.iloc[ind, :], num_top_restaurants)

neighborhoods_restaurants_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Italian Restaurant,Japanese Restaurant,Chinese Restaurant,Mexican Restaurant,Sushi Restaurant,Vietnamese Restaurant,Filipino Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant
1,Carnegie Hill,French Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Japanese Restaurant,American Restaurant,Thai Restaurant,Indian Restaurant,Argentinian Restaurant,Sushi Restaurant,Kosher Restaurant
2,Central Harlem,French Restaurant,African Restaurant,American Restaurant,Seafood Restaurant,Chinese Restaurant,Southern / Soul Food Restaurant,Ethiopian Restaurant,Restaurant,Caribbean Restaurant,Falafel Restaurant
3,Chelsea,American Restaurant,French Restaurant,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Sushi Restaurant,New American Restaurant,Chinese Restaurant,Paella Restaurant,Indian Restaurant
4,Chinatown,Chinese Restaurant,Vietnamese Restaurant,American Restaurant,Shanghai Restaurant,Greek Restaurant,Malay Restaurant,Mexican Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Hotpot Restaurant


### Cluster Neighborhoods using k-means clustering

In [84]:
# Run k-means to cluster the neighborhood into 7 clusters.

# set number of clusters
kclusters = 7

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([6, 4, 4, 4, 2, 4, 4, 6, 2, 0], dtype=int32)

In [85]:
# add clustering labels
neighborhoods_restaurants_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_restaurants_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.tail() # check the last rows!

manhattan_merged = manhattan_merged.dropna() # drop NaN clusters


In [86]:
manhattan_merged.shape

(39, 15)

In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### After Examination, I concluded that cluster # 0 should be the area for the new Restaurant as in this cluster Italian Restaurants are mostly 1st most common venue

In [88]:
best_cluster = manhattan_merged[manhattan_merged['Cluster Labels'] == 0]
best_cluster.shape

(13, 15)

### We focus only on the area where Italian Restaurants are 1st most common venue

In [89]:
italian_restaurants = best_cluster[best_cluster.iloc[:,5] == "Italian Restaurant"]
italian_restaurants.shape

(13, 15)

In [65]:
# create map of Manhattan using latitude and longitude values
map_it_restaurants = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(italian_restaurants['Latitude'], italian_restaurants['Longitude'], italian_restaurants['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_it_restaurants)  
    
map_it_restaurants

## 5. Results and Discussion <a name="results"></a>

We were able to identify and cluster in seven groups the location of Restaurants in Manhattan.

The results from our analysis shows that Italian Restaurants are first most common venues in many Neighborhoods in Manhattan. 

After the clustering, we focused only on Italian Restaurants that are first most common venues and we noticed that they are located in “Lower Manhattan” and near to the “East River” area. 
The areas mentioned above will be considered as a starting point for a more detailed analysis like competition, available properties and renting price. 


## 6. Conclusion <a name="conclusion"></a>

The purpose of this project was to help e friend to find Neighborhoods in Manhattan where Italian Restaurants are very popular and people living and working there like Italian cuisine. 
By using Foursquare API we were able to identify and to sort Restaurants in Manhattan and their respective Neighborhood. 
We clustered the locations in seven groups in order to identify the areas where Italian Restaurants are first most common venues. W
e concluded that most prominent Italian Restaurants are located in Neighborhoods like Little Italy, Financial District, Noho, Greenwich Village, etc. 
My friend will make the final decision after he further analyses other characteristics of the above Neighborhoods. 
